## Server MAS
Adapted by [Jorge Cruz](https://jcrvz.co) for:
TC2008B. Sistemas Multiagentes y Gráficas Computacionales. Tecnológico de Monterrey.

> Revised version, Nov. 2021

> Original implementation: Python server to interact with Unity, Sergio Ruiz, Jul. 2021

In [2]:
# Load the local driver (you need to activate your Drive in Colab)
%cd "/content/drive/MyDrive/TC2008B/G4/"

# Size of the board:
width, height = 50, 50

In [6]:
# Install pyngrok to propagate the http server
# %pip install pyngrok --quiet

# Load the required packages
from pyngrok import ngrok
from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import json
import os
import numpy as np

# Boid is a dummy implementation that can be replaced with a more sophisticated 
# MAS module, for example, MESA
from boid import Boid

# Start ngrok
ngrok.install_ngrok()

# Terminate open tunnels if exist
ngrok.kill()

# Open an HTTPs tunnel on port 8585 for http://localhost:8585
port = os.environ.get("PORT", 8585)
server_address = ("", port)

public_url = ngrok.connect(port="8585", proto="http", options={"bind_tls": True})
print("\n" + "#" * 94)
print(f"## Tracking URL: {public_url} ##")
print("#" * 94, end="\n\n")

PyngrokNgrokInstallError: An error occurred while downloading ngrok from https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-darwin-amd64.zip: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)>

In [7]:
ngrok.kill()

# Set the number of agents here:
num_agents = 30

flock = [Boid(*np.random.rand(2) * num_agents, width, height) 
  for _ in range(num_agents)]

flock_class = ['Green', 'Red'] * (30 // 2)
np.random.shuffle(flock_class)

# The way how agents are updated (per step/iteration)
def updateFeatures():
    global flock
    features = []

    # For each agent...
    for boid, colour in zip(flock, flock_class):
        # Update its state
        boid.apply_behaviour(flock)
        boid.update()

        # Read its features
        position = list(boid.edges())        
        features.append([{"x": position[0], "y": position[2], "z": position[1]},
                         'Ball', colour])

    return features

# Post the information in `features` for each iteration
def featuresToJSON(info_list):
    featureDICT = []
    for info in info_list:
        feature = {
            "position" : info[0], # position
            "kind" : info[1], # kind
            "colour" : info[2] # colour
        }
        featureDICT.append(feature)
    return json.dumps(featureDICT)

# This is the server. It controls the simulation.
# Server run (do not change it)
class Server(BaseHTTPRequestHandler):
    
    def _set_response(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        
    def do_GET(self):
        logging.info("GET request,\nPath: %s\nHeaders:\n%s\n", 
                     str(self.path), str(self.headers))
        self._set_response()
        self.wfile.write("GET request for {}".format(self.path).encode('utf-8'))

    def do_POST(self):
        content_length = int(self.headers['Content-Length'])

        #post_data = self.rfile.read(content_length)
        post_data = json.loads(self.rfile.read(content_length))
        
        # If you have issues with the encoder, toggle the following lines: 
        #logging.info("POST request,\nPath: %s\nHeaders:\n%s\n\nBody:\n%s\n",
                     #str(self.path), str(self.headers), post_data.decode('utf-8'))
        logging.info("POST request,\nPath: %s\nHeaders:\n%s\n\nBody:\n%s\n",
                     str(self.path), str(self.headers), json.dumps(post_data))

        # Here, magick happens 
        # --------------------       
        features = updateFeatures()
        #print(features)

        self._set_response()
        resp = "{\"data\":" + featuresToJSON(features) + "}"
        #print(resp)

        self.wfile.write(resp.encode('utf-8'))

# Server run (do not change it)
def run(server_class=HTTPServer, handler_class=Server, port=8585):
    logging.basicConfig(level=logging.INFO)
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)

    public_url = ngrok.connect(port).public_url
    logging.info("ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(
        public_url, port))

    logging.info("Starting httpd...\n") # HTTPD is HTTP Daemon!
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:   # CTRL + C stops the server
        pass

    httpd.server_close()
    logging.info("Stopping httpd...\n")

In [8]:
run(HTTPServer, Server)

INFO:pyngrok.ngrok:Opening tunnel named: http-8585-2d80fb63-267c-4e06-8cc7-1860838e73fd


PyngrokNgrokInstallError: An error occurred while downloading ngrok from https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-darwin-amd64.zip: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)>